Paso 1: Importaciones y configuración

In [17]:
import os
import glob
import rasterio
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from rasterio.plot import show
import fiona, time
from rasterio.plot import plotting_extent
from rasterio.warp import reproject, Resampling
from rasterio.enums import Resampling

# Configuración de rutas relativas (asumiendo que el notebook está en /notebooks)
RAW_DIR = "../data/raw"
PREPROCESING_DIR = "../data/processed"
RASTER_DIR = "../outputs/figures/raster/processed"
os.makedirs(RASTER_DIR, exist_ok=True)

print("✅ Librerías importadas y rutas configuradas.")

✅ Librerías importadas y rutas configuradas.


In [18]:
# Diccionario con tus archivos raster clave
rasters = {
    "Viento (ERA5)": os.path.join(RAW_DIR, "era5_wind_valdivia.tif"),
    "Vegetación (Sentinel-2)": os.path.join(RAW_DIR, "sentinel2_valdivia.tif"),
    "Topología (SRTM)": os.path.join(RAW_DIR, "srtm_valdivia.tif")
}

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

ensure_dir(PREPROCESING_DIR)

def save_raster(path, array, profile):
    profile_out = profile.copy()
    profile_out.update(count=array.shape[0])
    with rasterio.open(path, 'w', **profile_out) as dst:
        dst.write(array)

# ---------------------------------------------------
# --------- FUNCIÓN PARA NORMALIZAR REFLECTANCIA ----
# ---------------------------------------------------

def normalize_if_needed(band):
    """Detecta si una banda está normalizada y la normaliza a rango [0,1] si es necesario."""

    bmin, bmax = np.nanmin(band), np.nanmax(band)
    print(f"   → Banda min={bmin:.3f}, max={bmax:.3f}")

    # Caso A: los valores ya están entre 0 y 1
    if 0 <= bmin >= -0.01 and 0 <= bmax <= 1.5:
        print("   ✓ Banda YA está normalizada.")
        return band

    # Caso B: Sentinel-2 típico (0–10000)
    if bmax > 1 and bmax <= 20000:
        print("   ✓ Normalizando banda (escala 0–10000 → 0–1).")
        return band / 10000.0

    # Caso C: TIFF uint16 completo (0–65535)
    if bmax > 20000:
        print("   ✓ Normalizando banda (escala 0–65535 → 0–1).")
        return band / 65535.0

    print("   ⚠ Banda no reconocida, intentando normalizar por máximo detectado.")
    return band / bmax


# ---------------------------------------------------
# --------------------- PROCESAMIENTO ----------------
# ---------------------------------------------------

for nombre, ruta in rasters.items():

    print(f"\nProcesando: {nombre}")

    if not os.path.exists(ruta):
        print(f"❌ No encontrado: {ruta}")
        continue
    
    with rasterio.open(ruta) as src:
        data = src.read().astype(float)
        profile = src.profile

    # ============================================================
    # ----------------------  VEGETACIÓN S2 -----------------------
    # ============================================================
    if "Vegetación" in nombre:

        print("\n--- Normalización Sentinel-2 ---")

        # Sentinel-2: bandas uint16 → normalizar a [0,1]
        red   = normalize_if_needed(data[0])
        green = normalize_if_needed(data[1])
        blue  = normalize_if_needed(data[2])
        nir   = normalize_if_needed(data[3])

        # NDVI
        ndvi = (nir - red) / (nir + red + 1e-6)

        # Clasificación NDVI
        ndvi_class = np.zeros_like(ndvi, dtype=np.uint8)
        ndvi_class[(ndvi >= -1) & (ndvi < 0.1)] = 1   # suelo / no vegetación
        ndvi_class[(ndvi >= 0.1) & (ndvi < 0.3)] = 2  # vegetación baja
        ndvi_class[(ndvi >= 0.3) & (ndvi < 0.5)] = 3  # vegetación media
        ndvi_class[(ndvi >= 0.5)] = 4                 # vegetación alta

        processed = np.stack([red, green, blue, nir, ndvi, ndvi_class])

        profile_out = profile.copy()
        profile_out.update(dtype=rasterio.float32, count=processed.shape[0])

        save_raster(os.path.join(PREPROCESING_DIR, "vegetacion_processed.tif"),
                    processed.astype(np.float32), profile_out)

        print("✓ Vegetación procesada.")

    # ============================================================
    # -----------------------  VIENTO ERA5 ------------------------
    # ============================================================
    elif "Viento" in nombre:

        u, v = data
        mag = np.sqrt(u**2 + v**2)
        direc = (np.arctan2(v, u) * 180 / np.pi + 360) % 360

        processed = np.stack([u, v, mag, direc])

        profile_out = profile.copy()
        profile_out.update(dtype=rasterio.float32, count=processed.shape[0])

        save_raster(os.path.join(PREPROCESING_DIR, "viento_processed.tif"),
                    processed.astype(np.float32), profile_out)

        print("✓ Viento procesado.")

    # ============================================================
    # -----------------------  TOPOGRAFÍA SRTM --------------------
    # ============================================================
    elif "Topología" in nombre:

        elev = data[0]

        dy, dx = np.gradient(elev.astype(float), 
                             profile["transform"][4], 
                             profile["transform"][0])

        # Pendiente en grados (0–90°)
        slope = np.arctan(np.sqrt(dx**2 + dy**2)) * 180 / np.pi

        # Aspecto (orientación de la pendiente)
        aspect = np.arctan2(dx, -dy) * 180 / np.pi
        aspect = (aspect + 360) % 360

        processed = np.stack([elev, slope, aspect])

        profile_out = profile.copy()
        profile_out.update(dtype=rasterio.float32, count=processed.shape[0])

        save_raster(os.path.join(PREPROCESING_DIR, "topologia_processed.tif"),
                    processed.astype(np.float32), profile_out)

        print("✓ Topografía procesada.")

print("\n=== PREPROCESAMIENTO COMPLETO ===")
print("Datos guardados en:", PREPROCESING_DIR)


Procesando: Viento (ERA5)
✓ Viento procesado.

Procesando: Vegetación (Sentinel-2)

--- Normalización Sentinel-2 ---
   → Banda min=960.000, max=9600.000
   ✓ Normalizando banda (escala 0–10000 → 0–1).
   → Banda min=987.000, max=10344.000
   ✓ Normalizando banda (escala 0–10000 → 0–1).
   → Banda min=999.000, max=9408.000
   ✓ Normalizando banda (escala 0–10000 → 0–1).
   → Banda min=891.000, max=9648.000
   ✓ Normalizando banda (escala 0–10000 → 0–1).
✓ Vegetación procesada.

Procesando: Topología (SRTM)
✓ Topografía procesada.

=== PREPROCESAMIENTO COMPLETO ===
Datos guardados en: ../data/processed


In [20]:
# === Función para guardar figuras ===
def save_plot(fig, filename):
    out_path = os.path.join(RASTER_DIR, filename)
    fig.savefig(out_path, dpi=200, bbox_inches="tight")
    plt.close(fig)
    print(f"✓ Guardado: {out_path}")


# ============================================================
# ---------------------- VEGETACIÓN S2 ------------------------
# ============================================================

veg_path = os.path.join(PREPROCESING_DIR, "vegetacion_processed.tif")

if os.path.exists(veg_path):
    
    with rasterio.open(veg_path) as src:
        red, green, blue, nir, ndvi, ndvi_class = src.read()

    # ---------------------- RGB ----------------------
    rgb = np.stack([red, green, blue], axis=-1)
    rgb = np.clip(rgb, 0, 1)

    fig = plt.figure(figsize=(8, 8))
    plt.imshow(rgb)
    plt.title("RGB (normalizado) – Sentinel-2")
    plt.axis("off")
    save_plot(fig, "vegetacion_rgb.png")

    # ---------------------- NDVI MAP ----------------------
    fig = plt.figure(figsize=(8, 8))
    plt.imshow(ndvi, cmap="viridis", vmin=-1, vmax=1)
    plt.colorbar(label="NDVI")
    plt.title("Mapa NDVI – Sentinel-2")
    plt.axis("off")
    save_plot(fig, "vegetacion_ndvi_map.png")

    # ---------------------- NDVI HISTOGRAM ----------------------
    fig = plt.figure(figsize=(8, 5))
    plt.hist(ndvi.ravel(), bins=50, color="gray")
    plt.title("Histograma NDVI – Sentinel-2")
    plt.xlabel("NDVI")
    plt.ylabel("Frecuencia")
    save_plot(fig, "vegetacion_ndvi_hist.png")

    # ---------------------- NDVI CLASIFICADO ----------------------
    ndvi_colors = np.zeros((*ndvi_class.shape, 3))

    ndvi_colors[ndvi_class == 1] = [0.75, 0.75, 0.75]  # Gris – Suelo
    ndvi_colors[ndvi_class == 2] = [1.0, 1.0, 0.4]     # Amarillo – Vegetación baja
    ndvi_colors[ndvi_class == 3] = [0.4, 1.0, 0.4]     # Verde medio
    ndvi_colors[ndvi_class == 4] = [0.0, 0.6, 0.0]     # Verde oscuro

    fig = plt.figure(figsize=(8, 8))
    plt.imshow(ndvi_colors)
    plt.title("NDVI Clasificado – Sentinel-2")
    plt.axis("off")
    save_plot(fig, "vegetacion_ndvi_class.png")

else:
    print("⚠ No se encontró vegetacion_processed.tif — saltando plots de vegetación.")


# ============================================================
# ---------------------- VIENTO ERA5 --------------------------
# ============================================================

viento_path = os.path.join(PREPROCESING_DIR, "viento_processed.tif")

if os.path.exists(viento_path):

    with rasterio.open(viento_path) as src:
        u, v, mag, direc = src.read()

    # ---------------- Magnitud del viento ----------------
    fig = plt.figure(figsize=(8, 8))
    plt.imshow(mag, cmap="viridis")
    plt.title("Magnitud del Viento (ERA5)")
    plt.colorbar(label="m/s")
    plt.axis("off")
    save_plot(fig, "viento_magnitud.png")

    # ---------------- Dirección del viento ----------------
    fig = plt.figure(figsize=(8, 8))
    plt.imshow(direc, cmap="hsv")
    plt.title("Dirección del Viento (ERA5)")
    plt.colorbar(label="Grados")
    plt.axis("off")
    save_plot(fig, "viento_direccion.png")

else:
    print("⚠ No se encontró viento_processed.tif — saltando plots de viento.")


# ============================================================
# ---------------------- TOPOGRAFÍA SRTM ---------------------
# ============================================================

topo_path = os.path.join(PREPROCESING_DIR, "topologia_processed.tif")

if os.path.exists(topo_path):

    with rasterio.open(topo_path) as src:
        elev, slope, aspect = src.read()

    # ---------------- Elevación ----------------
    fig = plt.figure(figsize=(8, 8))
    plt.imshow(elev, cmap="terrain")
    plt.title("Elevación – SRTM")
    plt.colorbar(label="m")
    plt.axis("off")
    save_plot(fig, "topografia_elevacion.png")

    # ---------------- Pendiente ----------------
    fig = plt.figure(figsize=(8, 8))
    plt.imshow(slope, cmap="magma")
    plt.title("Pendiente (grados) – SRTM")
    plt.colorbar(label="°")
    plt.axis("off")
    save_plot(fig, "topografia_pendiente.png")

    # ---------------- Aspecto ----------------
    fig = plt.figure(figsize=(8, 8))
    plt.imshow(aspect, cmap="hsv")
    plt.title("Orientación (aspect) – SRTM")
    plt.colorbar(label="°")
    plt.axis("off")
    save_plot(fig, "topografia_orientacion.png")

else:
    print("⚠ No se encontró topologia_processed.tif — saltando plots de topografía.")


print("\n=== FIGURAS GENERADAS Y GUARDADAS EN:", RASTER_DIR, "===")

✓ Guardado: ../outputs/figures/raster/processed\vegetacion_rgb.png
✓ Guardado: ../outputs/figures/raster/processed\vegetacion_ndvi_map.png
✓ Guardado: ../outputs/figures/raster/processed\vegetacion_ndvi_hist.png
✓ Guardado: ../outputs/figures/raster/processed\vegetacion_ndvi_class.png
✓ Guardado: ../outputs/figures/raster/processed\viento_magnitud.png
✓ Guardado: ../outputs/figures/raster/processed\viento_direccion.png
✓ Guardado: ../outputs/figures/raster/processed\topografia_elevacion.png
✓ Guardado: ../outputs/figures/raster/processed\topografia_pendiente.png
✓ Guardado: ../outputs/figures/raster/processed\topografia_orientacion.png

=== FIGURAS GENERADAS Y GUARDADAS EN: ../outputs/figures/raster/processed ===
